# Import Data Set

In [ ]:
import pandas as pd
data = pd.read_csv("WC_AT.csv")
data.head()

In [ ]:
import pandas_profiling as pp
EDA_report= pp.ProfileReport(data)
EDA_report.to_file(output_file='wcATreport.html')

In [ ]:
data.info()

# Correlation

In [ ]:
import seaborn as sns
%matplotlib inline
sns.pairplot(data)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter("Waist","AT",data=data)
#heteroscedasticity

In [ ]:
data.corr()



In [ ]:
import seaborn as sns
sns.distplot(data['Waist'])



In [ ]:
import seaborn as sns
sns.distplot(data['AT'])

## Fitting a Linear Regression Model

In [ ]:

sns.boxplot(data.Waist)

In [ ]:
import statsmodels.formula.api as smf
model_basic=smf.ols("AT~Waist",data=data).fit()
model_basic.summary()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
x=np.log(data.Waist)
y=data.AT
plt.scatter(x,y)
plt.xlabel("log(Waist)")
plt.ylabel("AT")

In [ ]:
import statsmodels.formula.api as smf
data["logwaist"]=np.log(data.Waist)
#model with log transformation
model_log=smf.ols("AT~logwaist",data=data).fit()
model_log.summary()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
x=data.Waist
y=np.log(data.AT)
plt.scatter(x,y)
plt.xlabel("Waist")
plt.ylabel("logAT")
data.corr()

In [ ]:
# Exponential model
data["log_AT"]=np.log(data.AT)# creating a column for log of AT

#model with log_at

model_exp=smf.ols("log_AT~Waist",data=data).fit()
model_exp.summary()


In [ ]:
sns.regplot(x="Waist", y="log_AT",data=data);

In [ ]:
pred=np.exp(model_exp.predict(data.Waist))
pred

In [ ]:
errors=data.AT-pred
errors.mean()

In [ ]:
# As from the above visualization we can observe that log of AT and waist are having a curvilinear relation
# Model with Quadtratic equation y= bo+b1x+b2x^2
# y=a+bx+cx2
data["waist_sq"]=data.Waist*data.Waist
print(data.corr())
model_quad=smf.ols("log_AT~Waist+waist_sq",data=data).fit()
model_quad.summary()


In [ ]:
#calculating RMSE
def RMSE(actual,pred):
    temp=np.sqrt(np.mean((actual-pred)*(actual-pred)))
    return temp

In [ ]:
#RMSE for model_basic

pred_basic=model_basic.predict(data)
rmse_basic=RMSE(data.AT,pred_basic)
rmse_basic



In [ ]:
#RMSE for model_log

pred_log=model_log.predict(data)
rmse_log=RMSE(data.AT,pred_log)
rmse_log

In [ ]:
#RMSE for model_exp

pred_exp=np.exp(model_exp.predict(data))
rmse_exp=RMSE(data.AT,pred_exp)
rmse_exp

In [ ]:
#RMSE for model_quad

pred_quad=np.exp(model_quad.predict(data))
rmse_quad=RMSE(data.AT,pred_quad)
rmse_quad

In [ ]:
model_basic.aic

In [ ]:
dict={"model":["basic","log","exp","quad"], "RMSE":[rmse_basic,rmse_log,rmse_exp,rmse_quad],"AIC":[model_basic.aic,model_log.aic,model_exp.aic,model_quad.aic]}
rmse_table=pd.DataFrame(dict)
rmse_table

In [ ]:
#Coefficients
model_quad.params


In [ ]:
#t and p-Values
print(model_quad.tvalues, '\n', model_quad.pvalues)    

In [ ]:
#R squared values
(model_quad.rsquared,model_quad.rsquared_adj)

# Predict for new data point

In [ ]:
#Predict for 165 Waist
newdata=pd.Series([76,101])

In [ ]:
data_pred=pd.DataFrame(newdata,columns=['Waist'])
data_pred["waist_sq"]=data_pred.Waist*data_pred.Waist
np.exp(model_quad.predict(data_pred))